# 工具

In [13]:
from openai import OpenAI

model_id = 'qwen/Qwen2-7B-Instruct-GGUF'

client = OpenAI(
    base_url='https://ms-fc-capp-func-bkdebcmpmr.cn-hangzhou.fcapp.run/v1',
    api_key='ollama'
)

response=client.chat.completions.create(
    model=model_id,
    messages=[{"role":"user", "content":"随机给我一个百家姓中的一个姓即可"}],
    stream=True
)

for chunk in response:
    print(chunk.choices[0].delta.content, end='')

曹

## 工具定义

In [3]:
from illufly.types import ToolAgent, TextBlock

def mytool(city: str):
    """查询天气可以找我"""
    yield TextBlock("chunk", f"{city}今天有雷雨")

tool = ToolAgent(mytool)
tool("广州")

广州今天有雷雨



In [1]:
from illufly.agent import ChatQwen, Tool
from illufly.io import log, TextBlock

def get_current_weather(location: str):
    """获取城市的天气情况"""
    # return f"{location}今天是晴天。 "
    yield TextBlock("chunk", f"{location}今天是晴天。 ")

mytool = Tool(get_current_weather)
log(mytool, "广州")

广州今天是晴天。 

''

In [2]:
mytool.tool

{'type': 'function',
 'function': {'name': 'get_current_weather',
  'description': '获取城市的天气情况',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'str', 'description': ''}},
   'required': ['location']}}}

## 提示大模型返回工具

In [16]:
from illufly.chat import ChatQwen
from illufly.types import ToolAgent
from illufly.io import log, TextBlock

def get_current_weather(location: str):
    """获取城市的天气情况"""
    # return f"{location}今天是晴天。 "
    yield TextBlock("chunk", f"{location}今天是晴天。 ")

mytool = ToolAgent(get_current_weather)

In [18]:
mytool.__class__.__name__

'ToolAgent'

In [2]:
mytool.tool_desc

{'type': 'function',
 'function': {'name': 'get_current_weather',
  'description': '获取城市的天气情况',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string', 'description': ''}},
   'required': ['location']}}}

In [3]:
qwen = ChatQwen(tools=[mytool])

template_text = f"""
你是强大的写作助理。

请根据我的任务要求返回需要执行的工具函数，可以返回一个、多个，也可以不返回。

可用的工具函数包括：
{qwen.get_tools_name()}

如果你决定返回工具函数描述，请遵循如下格式：
{qwen.get_tools_desc()}
{qwen.get_tools_instruction()}
"""

print(template_text)


你是强大的写作助理。

请根据我的任务要求返回需要执行的工具函数，可以返回一个、多个，也可以不返回。

可用的工具函数包括：
get_current_weather

如果你决定返回工具函数描述，请遵循如下格式：
[{'type': 'function', 'function': {'name': 'get_current_weather', 'description': '获取城市的天气情况', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': ''}}, 'required': ['location']}}}]
从列表 [get_current_weather] 中选择一个或多个funciton，并按照下面的格式输出函数描述列表，描述每个函数的名称和参数：
```json <tools-calling>
[{"index": "integer: index of selected function", "function": {"name": "(string): 填写选中工具的参数名称", "parameters": "(json): 填写具体参数值"}}]
```
**工具函数输出示例：**
```json
[{"index": 0, "function": {"name": "get_current_weather", "parameters": "{"location": "广州"}"}},
{"index": 1, "function": {"name": "get_current_weather", "parameters": "{"location": "上海"}"}}]
```



In [4]:
task = "请你帮指定周六带全家去上海游玩的计划"

template_text = f"""
For the following task, make plans that can solve the problem step by step. 
For each plan, indicate which external tool together with tool input to retrieve evidence. 
You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
{qwen.get_tools_desc()}

For example,

Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked xhours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hoursless than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solvewith Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x − 10) + ((2x − 10) − 8) = 157]
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 ∗ #E2 − 10) − 8]

Begin! Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}
"""

print(template_text)


For the following task, make plans that can solve the problem step by step. 
For each plan, indicate which external tool together with tool input to retrieve evidence. 
You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find shortand succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with generalworld knowledge and common sense. Prioritize it when you are confident in solving the problemyourself. Input can be any instruction.

For example,

Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked xhours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hoursless than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, trans

In [14]:
task = "请你帮我策划一个修仙小说大纲，200字"

template_text = f"""
1. For the following task, make plans that can solve the problem step by step. 
2. For each plan, indicate which external tool together with tool input to retrieve evidence. 
3. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)
4. 请帮我根据引用关系，整理工具的调用批次，将可以并行执行的工具放入同一批次

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find shortand succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with generalworld knowledge and common sense. Prioritize it when you are confident in solving the problemyourself. Input can be any instruction.

plan example,

Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked xhours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hoursless than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solvewith Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x − 10) + ((2x − 10) − 8) = 157]
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 ∗ #E2 − 10) − 8]

Begin! Describe your plans with rich details. 

Task: {task}

batch example,

#E1, #E2
#E3
"""

print(template_text)


1. For the following task, make plans that can solve the problem step by step. 
2. For each plan, indicate which external tool together with tool input to retrieve evidence. 
3. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)
4. 请帮我根据引用关系，整理工具的调用批次，将可以并行执行的工具放入同一批次

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find shortand succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with generalworld knowledge and common sense. Prioritize it when you are confident in solving the problemyourself. Input can be any instruction.

plan example,

Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked xhours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hoursless than Toby. How many hours did Rebe

In [15]:
q = ChatQwen(memory=template_text)
log(q, "请开始")

Plan: 首先，通过Google搜索了解当前最热门的修仙小说题材和元素。#E1 = Google[当前最热门的修仙小说题材]

Plan: 根据搜索结果，结合通用的世界构建知识，构思一个新颖而吸引人的背景设定。#E2 = LLM[基于#E1，创造一个独特的修仙世界背景]

Plan: 设计主角及其性格特点，确保其具有成长性并与背景设定紧密结合。#E3 = LLM[创造一个与#E2世界相匹配，具有鲜明个性和成长空间的主角]

Plan: 确定小说中的主要冲突与挑战，这些应反映出修仙世界的残酷法则及主角的成长路径。#E4 = LLM[基于#E2和#E3，设计小说的主要冲突和修行挑战]

Plan: 创造几个关键的配角，包括盟友和对手，他们的存在需推动故事发展并深化主题。#E5 = LLM[围绕#E3的主角，设计几个有深度、能促进情节发展的配角角色]

Plan: 概括小说的起承转合，确保剧情紧凑且高潮迭起，同时体现修仙小说特有的修炼升级体系。#E6 = LLM[整合#E2至#E5的内容，编写一个包含起始、发展、转折和结局的简要大纲，约200字]

#E1
#E2, #E3
#E4
#E5
#E6



In [ ]:
solve_prompt = """
Solve the following task or problem. 
To solve the problem, we have made step-by-step Plan and retrieved corresponding Evidence to each Plan. 
Use them with caution since long evidence might contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. 
Respond with the answerdirectly with no extra words.

Task: {task}
Response:
"""

In [5]:
q = ChatQwen(memory=template_text)
log(q, "帮我根据北京和巴黎的天气写一首四句儿歌")

为了创作这首儿歌，我首先需要获取北京和巴黎当前的天气情况。之后，我会根据这两个城市的天气信息来撰写儿歌。让我们先调用天气查询工具。

```json
[{"index": 0, "function": {"name": "get_current_weather", "parameters": "{\"location\": \"北京\""}}},
{"index": 1, "function": {"name": "get_current_weather", "parameters": "{\"location\": \"巴黎\""}}}]
```

'为了创作这首儿歌，我首先需要获取北京和巴黎当前的天气情况。之后，我会根据这两个城市的天气信息来撰写儿歌。让我们先调用天气查询工具。\n\n```json\n[{"index": 0, "function": {"name": "get_current_weather", "parameters": "{\\"location\\": \\"北京\\""}}},\n{"index": 1, "function": {"name": "get_current_weather", "parameters": "{\\"location\\": \\"巴黎\\""}}}]\n```'

In [6]:
q = ChatQwen(memory=template_text)
log(q, "帮我写一首四句儿歌")

当然可以，这里有一首简单温馨的四句儿歌供您参考：

小雨滴滴答，落在花园里。
花朵轻轻摇，笑得甜蜜蜜。

无需调用工具函数，直接提供内容即可。

'当然可以，这里有一首简单温馨的四句儿歌供您参考：\n\n小雨滴滴答，落在花园里。\n花朵轻轻摇，笑得甜蜜蜜。\n\n无需调用工具函数，直接提供内容即可。'

In [ ]:
[
    {
        'role': 'user',
        'content': '今天广州可以踏青吗'},
    {
        'role': 'assistant',
        'content': '',
        'tool_calls': '{
            "0": {
                "index": 0, "id": "call_128cf28b709847ba8e8369",
                "type": "function",
                "function": {
                    "name": "get_current_weather",
                    "arguments": "{\\"location\\": \\"广州\\"}"
                }}
                  }'
                  },
                  {'tool_call_id': 'call_128cf28b709847ba8e8369', 'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天是晴天。 '}]

## 工具引用

In [3]:
q = ChatQwen(tools=[mytool])
log(q, "今天广州可以踏青吗", verbose=True)
q.memory

new_memory [{'role': 'user', 'content': '今天广州可以踏青吗'}]
[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_128cf28b709847ba8e8369", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_128cf28b709847ba8e8369", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
广州今天是晴天。 new_memory [{'role': 'user', 'content': '今天广州可以踏青吗'}, {'role': 'assistant', 'content': '', 'tool_calls': '{"0": {"index": 0, "id": "call_128cf28b709847ba8e8369", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\\"location\\": \\"广州\\"}"}}}'}, {'tool_call_id': 'call_128cf28b709847ba8e8369'

[{'role': 'user', 'content': '今天广州可以踏青吗'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': '{"0": {"index": 0, "id": "call_128cf28b709847ba8e8369", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\\"location\\": \\"广州\\"}"}}}'},
 {'tool_call_id': 'call_128cf28b709847ba8e8369',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '广州今天是晴天。 '}]

In [4]:
q = ChatQwen(name="查看天气", enable_search=True)
q.tool
log(ChatQwen(), "今天广州可以踏青吗", tools=[t.tool for t in [q]], toolkits=[q], verbose=True)

[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_49981d40a2394273990f1f", "type": "function", "function": {"name": "查看天气", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"prompt\":\"广州"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "\",\"args\":\"\",\"kwargs\":\"\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_49981d40a2394273990f1f", "type": "function", "function": {"name": "查看天气", "arguments": "{\"prompt\":\"广州\",\"args\":\"\",\"kwargs\":\"\"}"}}}
广州，简称“穗”，别称羊城、花城，是广东省省会、副省级市、国家中心城市、超大城市，国务院批复确定的中国重要的中心城市、国际商贸中心和综合交通枢纽。广州地处中国南部、珠江下游、濒临南海，是中国南部战区司令部驻地，国家物流枢纽，国家综合性门户城市，首批沿海开放城市，是中国通往世界的南大门，是粤港澳大湾区、泛珠江三角洲经济区的核心城市以及一带一路的枢纽城市。

广州历史悠久，是首批国家历史文化名城之一，是广府文化的发祥地，也是岭南文化的重要组成部分。广州有着“食在广州”的美誉，是粤菜的发源地之一，同时拥有众多的名胜古迹和人文景观，如白云山、广州塔（小蛮腰）、上下

'广州今天的天气情况如下：实时天气为阴，湿度较高，达到95%，体感温度为29.0摄氏度。吹北风，风力较弱，为1级。根据之前的信息，今天白天的天气预报提及有中雨，气温在25.0至35.0摄氏度之间，与实时情况有所差异，可能是实际天气有所变化。由于存在不一致，建议结合最新发布的天气预报进行适时调整。另外，空气质量和能见度良好，空气质量指数为36，属于优质空气范围。请根据天气变化适当调整着装，并备好雨具出行。广州今天的天气有些阴并预报有中雨，气温在25.0至35.0摄氏度之间，湿度非常高，达到了95%。虽然雨水可能为踏青带来不便，但如果你喜欢雨中漫步，记得带上雨具，穿着防滑的鞋子，并选择合适的地点进行活动。考虑到体感温度较高，建议穿着轻薄透气的衣物，并注意补水。另外，目前空气质量很好，适合户外活动。请持续关注最新的天气预报，以便更好地安排您的踏青计划。'

## Tools Calling

仍然采纳`langchain`来定义工具，这可以大大简化回调工具的管理。

涉及定义工具的技巧主要包括：
- 使用 tool 装饰函数将新函数定义为工具类型
- 使用 StructTool 转换已有的函数为工具类型
- 使用 BaseModel 补充工具的参数定义

采用以上方法定义好工具，就可以使用 convert_to_openai_tool 将其转换为官方要求的 JSON 结构（确实会比手写节省很多精力）。

In [4]:
from illufly.tools import tool, convert_to_openai_tool
convert_to_openai_tool(get_current_weather)

{'type': 'function',
 'function': {'name': 'get_current_weather',
  'description': '获取城市的天气情况',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string'}},
   'required': ['location']}}}

In [1]:
from illufly.agent import Tool, ChatQwen
from illufly.io import log

def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

log(ChatQwen(), "今天广州天气如何啊", tools=[Tool(get_current_weather)], exec_tool=False, verbose=True)

[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_82adacb231a14c68a35f3d", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_82adacb231a14c68a35f3d", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}


'{"0": {"index": 0, "id": "call_82adacb231a14c68a35f3d", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\\"location\\": \\"广州\\"}"}}}'

In [3]:
from illufly.agent import Tool, ChatQwen
from illufly.io import log

def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

log(ChatQwen(), "今天广州天气可以晒被子吗", tools=[Tool(get_current_weather)], exec_tool=True, verbose=True)

[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_97b44bb7dfae4f678e5c3a", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_97b44bb7dfae4f678e5c3a", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
广州今天是晴天。 今天广州是晴天，非常适合晒被子。

'今天广州是晴天，非常适合晒被子。'

In [3]:
import json

json.dumps([{'role': 'user', 'content': '今天广州天气如何啊'}, 
{'role': 'assistant', 
 'content': '', 
 'tool_calls': {
   0:  {
    'index': 0, 
    'id': 'call_31a5aae2c7094a9da4c183', 
    'type': 'function', 
    'function': {
      'name': 'get_current_weather', 
      'arguments': '{\"location\": \"广州\"}'
      }
    }
 }
},
{
 'tool_call_id': 'call_31a5aae2c7094a9da4c183', 
 'role': 'tool', 
 'name': 'get_current_weather', 
 'content': '广州今天是晴天。 '
}])

'[{"role": "user", "content": "\\u4eca\\u5929\\u5e7f\\u5dde\\u5929\\u6c14\\u5982\\u4f55\\u554a"}, {"role": "assistant", "content": "", "tool_calls": {"0": {"index": 0, "id": "call_31a5aae2c7094a9da4c183", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\\"location\\": \\"\\u5e7f\\u5dde\\"}"}}}}, {"tool_call_id": "call_31a5aae2c7094a9da4c183", "role": "tool", "name": "get_current_weather", "content": "\\u5e7f\\u5dde\\u4eca\\u5929\\u662f\\u6674\\u5929\\u3002 "}]'

<div class="alert alert-info">
⚠️ 上面的调用仅仅是返回了工具描述，尚未真正执行工具代码。
</div>

## 支持数据分析的工具

In [1]:
import pandas as pd
from illufly.agent import Dataset, ChatQwen
from illufly.tools import create_python_code_tool
from illufly.io import log

data = {
    "考试成绩": Dataset(
        df=pd.DataFrame({
            "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
            "语文": [90, 80, 105, 110],
            "数学": [135, 110, 120, 90]
        }),
        desc="考试成绩"
    )
}
mytool = create_python_code_tool(data, ChatQwen())

# mytool
for x in mytool.func({"question": "赖国良成绩如何？"}):
    print(x)

```
python


def main():
    #
 从数据集中筛选出赖国
良的成绩
    lai_guo
_liang_scores = data['考试成绩
'].df[data['考试成绩'].df
['姓名'] == '赖国良
']
    # 返回赖国良的成绩

    return lai_guo_li
ang_scores[['语文', '数学']].
to_dict(orient='records')

```

```python
def main():
    # 从数据集中筛选出赖国良的成绩
    lai_guo_liang_scores = data['考试成绩'].df[data['考试成绩'].df['姓名'] == '赖国良']
    # 返回赖国良的成绩
    return lai_guo_liang_scores[['语文', '数学']].to_dict(orient='records')

```
语文: 105, 数学: 120


## 工作台 + 数据分析

### 多轮对话

In [1]:
from illufly.agent import ChatQwen, FakeLLM
from illufly.io import log, alog

import pandas as pd

a = ChatQwen()

a.state.add_dataset(
    "考试成绩",
    pd.DataFrame({
        "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
        "语文": [90, 80, 105, 110],
        "数学": [135, 110, 120, 90]
    })
)

In [4]:
await alog(a, "请使用工具查询，看看谁的语文考试成绩最好？", new_chat=True)

```python
def main():
    # 加载考试成绩数据集
    exam_scores = data['考试成绩'].df
    
    # 确定语文成绩最好的学生
    best_chinese_student = exam_scores.loc[exam_scores['语文'].idxmax()]['姓名']
    
    return best_chinese_student

main()
```

语文考试成绩最好的是吴红兵。



'{"0": {"index": 0, "id": "call_31b1904bb11d40eaaab906", "type": "function", "function": {"name": "python_code", "arguments": "{\\"question\\": \\"谁的语文考试成绩最好？\\"}"}}}语文考试成绩最好的是吴红兵。'

In [5]:
log(a, "看看他的总分第几名？")

```python
import pandas as pd

def main():
    global data
    # Load the exam scores dataset
    exam_scores_df = data['考试成绩'].df
    
    # Add a column for total score
    exam_scores_df['总分'] = exam_scores_df['语文'] + exam_scores_df['数学']
    
    # Sort the dataframe by total score in descending order
    sorted_scores_df = exam_scores_df.sort_values(by='总分', ascending=False)
    
    # Get Wu Hongbing's total score
    wu_hongbing_total = sorted_scores_df.loc[sorted_scores_df['姓名'] == '吴红兵', '总分'].iloc[0]
    
    # Get Wu Hongbing's rank
    wu_hongbing_rank = sorted_scores_df['总分'].eq(wu_hongbing_total).sum()
    
    return wu_hongbing_rank

main()
```

吴红兵的总分排名是第一名。



'{"0": {"index": 0, "id": "call_93556a2dfeb44eb282a993", "type": "function", "function": {"name": "python_code", "arguments": "{\\"question\\": \\"吴红兵的总分排名是多少？\\"}"}}}吴红兵的总分排名是第一名。'

In [6]:
log(a, "再帮我看看薛一凡的详细情况和名次")

```python
import pandas as pd

def rank_student(data, name):
    df = data['考试成绩'].df
    # 添加总分列（如果尚未添加）
    if '总分' not in df.columns:
        df['总分'] = df['语文'] + df['数学']
    # 按总分降序排序并添加排名列
    df_ranked = df.sort_values(by='总分', ascending=False).reset_index(drop=True)
    df_ranked['排名'] = df_ranked.index + 1
    # 查询指定学生的成绩信息及排名
    student_info = df_ranked[df_ranked['姓名'] == name]
    return student_info.to_dict(orient='records')

def main():
    result = rank_student(data, '薛一凡')
    return result
```

薛一凡的详细成绩信息如下：
- 语文: 90 分
- 数学: 135 分
总分为 225 分，排名是第一名。



'{"0": {"index": 0, "id": "call_7e5f7b34efe84b1681f560", "type": "function", "function": {"name": "python_code", "arguments": "{\\"question\\": \\"查询薛一凡的详细成绩信息及排名\\"}"}}}薛一凡的详细成绩信息如下：\n- 语文: 90 分\n- 数学: 135 分\n总分为 225 分，排名是第一名。'

### 单轮写作

In [8]:
from illufly.llm import qwen, fake_llm
from illufly.agent import Agent
from illufly.io import log
import pandas as pd

a = Agent(qwen)

a.add_dataset(
    "中秋名单",
    pd.DataFrame({
        "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
        "月饼数量": [2, 2, 1, 1]
    })
)

a.add_dataset(
    "考试成绩",
    pd.DataFrame({
        "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
        "语文": [90, 80, 105, 110],
        "数学": [135, 110, 120, 90]
    })
)

log(a, "请查询工具，针对这几个孩子的实际考试成绩，帮我写一份报告", template="OUTLINE")

[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_f35cc3756f604157a2223a", "type": "function", "function": {"name": "python_code", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"question\": \"查询"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "几个孩子的实际考试成绩\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_f35cc3756f604157a2223a", "type": "function", "function": {"name": "python_code", "arguments": "{\"question\": \"查询几个孩子的实际考试成绩\"}"}}}
```python
def main():
    # 由于问题没有具体说明要查询哪些孩子的成绩以及需要什么样的处理，
    # 这里我们做一个基本的假设：查询所有孩子的语文和数学成绩。
    # 如果有具体需求，请提供更详细的问题描述。

    # 使用pandas库读取考试成绩数据集
    exam_scores = data['考试成绩'].df
    
    # 返回所有孩子的语文和数学成绩
    return exam_scores[['姓名', '语文', '数学']]
```

[TOOL_RESP_FINAL] 
| 姓名   |   语文 |   数学 |
|

'# 学生考试成绩分析报告\n\n## 一、成绩概览\n\n### 成绩统计\n\n<OUTLINE>\n扩写摘要: 对学生的语文与数学成绩进行整体概括，列出平均分、最高分、最低分。\n\n扩写要求:\n- 计算并列举所有学生的语文和数学成绩的平均分。\n- 高出平均分最多的学生及其分数。\n- 低于平均分最多的学生及其分数。\n- 预估字数：150字\n</OUTLINE>\n\n## 二、个体成绩分析\n\n### 薛一凡\n\n<OUTLINE>\n扩写摘要: 分析薛一凡同学的成绩表现，与班级平均成绩对比。\n\n扩写要求:\n- 对比薛一凡的语文和数学成绩与班级相应科目的平均成绩。\n- 点评薛一凡成绩的优势科目及提升空间。\n- 预估字数：100字\n</OUTLINE>\n\n### 肖一笑\n\n<OUTLINE>\n扩写摘要: 评估肖一笑同学的学习情况，提出针对性建议。\n\n扩写要求:\n- 指出肖一笑成绩相对薄弱的科目。\n- 给出提高该科目成绩的具体建议。\n- 预估字数：100字\n</OUTLINE>\n\n### 赖国良\n\n<OUTLINE>\n扩写摘要: 分析赖国良的成绩分布，探讨其学习策略的有效性。\n\n扩写要求:\n- 分析赖国良在两门科目上的表现是否均衡。\n- 探讨赖国良维持或提升成绩可能采取的策略。\n- 预估字数：100字\n</OUTLINE>\n\n### 吴红兵\n\n<OUTLINE>\n扩写摘要: 评价吴红兵的成绩波动，提出平衡发展的重要性。\n\n扩写要求:\n- 说明吴红兵成绩的显著差异（高分与低分科目）。\n- 强调平衡各科学习的必要性，并提供策略。\n- 预估字数：100字\n</OUTLINE>\n\n## 三、总体建议与总结\n\n<OUTLINE>\n扩写摘要: 综合所有学生的表现，提出班级整体提升学习效率的方法。\n\n扩写要求:\n- 总结学生们的共性问题和个别亮点。\n- 提出适用于全班的一般性学习策略和方法。\n- 预估字数：150字\n</OUTLINE>'

### 扩写

In [18]:
log(a.from_outline(question="请使用工具查询，根据真实数据中的学生和成绩扩写"))

[INFO] 执行扩写任务 <5529-686-006>：
扩写摘要:
概述学生的整体成绩分布情况，包括最高分、最低分以及平均分数等。

扩写要求:
- 描述每个科目最高分与最低分的学生姓名及分数。
- 计算并提及每门科目的平均分。
- 预估字数: 150字
[TOOLS_CALL] {"index": 0, "id": "call_6e1711c3f15d4dc9b51b4d", "type": "function", "function": {"name": "python_code", "arguments": ""}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"question\": \"提供"}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "各科目的最高分、最低分"}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "学生姓名及分数，以及每门"}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "科目的平均分。\"}"}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_6e1711c3f15d4dc9b51b4d", "type": "function", "function": {"name": "python_code", "arguments": "{\"question\": \"提供各科目的最高分、最低分学生姓名及分数，以及每门科目

"```python\nimport pandas as pd\n\ndef calculate_subject_scores(df):\n    # 计算每门科目的最高分和最低分\n    max_scores = df.max(numeric_only=True)\n    min_scores = df.min(numeric_only=True)\n    \n    # 获取最高分和最低分的学生姓名\n    max_students = {subject: df[df[subject] == max_score]['姓名'].iloc[0] for subject, max_score in max_scores.items()}\n    min_students = {subject: df[df[subject] == min_score]['姓名'].iloc[0] for subject, min_score in min_scores.items()}\n    \n    # 计算每门科目的平均分\n    avg_scores = df.mean(numeric_only=True)\n    \n    # 返回结果\n    result = {\n        '最高分': max_students,\n        '最高分分数': max_scores,\n        '最低分': min_students,\n        '最低分分数': min_scores,\n        '平均分': avg_scores\n    }\n    return result\n\ndef main():\n    # 使用给定的数据集名称加载数据\n    dataset_name = '考试成绩'\n    data_df = data[dataset_name].df\n    \n    # 排除非成绩列（例如姓名）\n    scores_df = data_df.drop(columns=['姓名'])\n    \n    # 调用函数计算并返回结果\n    result = calculate_subject_scores(scores_df)\n    return result\n\nmain()\n`

In [19]:
a.state.from_outline

{'5529-686-006': [{'role': 'system',
   'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 你必须根据已有提纲扩写，不要修改提纲中对扩写的要求和限定，不要额外发挥\n\n**已有文字草稿如下:**\n```markdown\n# 学生考试成绩分析报告\n\n\n## 一、成绩概览\n\n\n### 1.1 成绩分布\n\n\n&lt;&lt;&lt;YOUR_TEXT&gt;&gt;&gt;\n\n\n### 1.2 科目比较\n\n\n...\n\n## 二、个体成绩分析\n\n\n### 2.1 优秀学生案例\n\n\n...\n\n### 2.2 待提升领域\n\n\n...\n\n## 三、总结与建议\n\n\n### 3.1 总体评价\n\n\n...\n\n### 3.2 教学策略建议\n\n\n...\n\n```\n\n**请你按照如下扩写任务要求生成一段文字，使其适合替换上面文字草稿中`<<<YOUR_TEXT>>>`所在位置:**\n```markdown\n扩写摘要:\n概述学生的整体成绩分布情况，包括最高分、最低分以及平均分数等。\n\n扩写要求:\n- 描述每个科目最高分与最低分的学生姓名及分数。\n- 计算并提及每门科目的平均分。\n- 预估字数: 150字\n```\n'},
  {'role': 'user', 'content': '请根据需要调用工具查询真实数据。'},
  {'role': 'assistant',
   'content': '',
   'tool_calls': [{'index': 0,
     'id': 'call_6e1711c3f15d4dc9b51b4d',
     'type': 'function',
     'function': {'name': 'python_code',
      'arguments': '{"question": "提供各科目的最高分、最低分学生姓名及分数，以及每门科目的平均分。"}'}}]},
  {'role': 'tool', 'name': 'python_code', 'conten

In [20]:
print(a.output)

# 学生考试成绩分析报告


## 一、成绩概览


### 1.1 成绩分布



- **数学**科目：最高分由肖一笑同学获得，得分135分；最低分出自吴红兵同学，得分为110分，该科平均分为122.5分。
- **语文**科目：薛一凡同学取得最高分90分，而赖国良同学的80分为最低分，科目平均分为85分。

以上数据显示了学生在不同科目上的成绩表现范围及整体水平。


### 1.2 科目比较



经分析，数学科目的整体成绩高于语文科目。数学平均分比语文高出5%，体现了学生在逻辑推理与计算能力上表现更佳。值得注意的是，李华同学在数学方面成绩显著，满分100分中获得98分，而其语文成绩则相对较低，仅为78分，显示了明显的偏科现象。这提示我们需关注并促进学生各科目均衡发展。


## 二、个体成绩分析


### 2.1 优秀学生案例



本次考试中，总分排名前三的学生展现了卓越的学术能力，具体情况如下：


1. **薛一凡**，以总分225分位居榜首，其中数学成绩高达135分，展现出其在数学领域的非凡天赋。

2. **赖国良**，同样取得225分的总分，数学120分与薛一凡并列第一，说明他在数学上同样有着出色的表现。

3. **吴红兵**，以总分200分位列第三，语文成绩尤其突出，达到110分，彰显了他在语言文学方面的深厚功底。

以上学生不仅总分名列前茅，各自在不同科目上的强势表现，为其他同学树立了学习的榜样。


### 2.2 待提升领域



经过细致分析，我们发现两位学生在特定科目上的表现显著低于班级平均分，具体情况如下：


- **学生A**在数学学科上得分远低于班级平均水平。建议学生A增加数学的日常练习量，重点关注代数与几何的解题技巧，同时参加课后辅导班或寻求家教个别辅导，以巩固基础概念并提升解题速度。

- **学生B**在英语科目上成绩滞后。针对学生B，建议加强英语阅读理解与词汇积累，利用英语学习软件每日练习听力与口语，加入英语角或语言交换伙伴项目以提升实际应用能力。

通过上述个性化提升计划，期待这两位学生能在下一学期展现出显著的进步。


## 三、总结与建议


### 3.1 总体评价



经过全面分析，本班级展现了多样化的学习成果，多数学生达到了课程基本要求。值得注意的是，学生间良好的团队合作氛围促进了知识共享与相互激励

## 图表

## 文生图

In [1]:
from illufly.tools import Text2ImageWanx
from illufly.io import log, alog

w = Text2ImageWanx()
log(w, "a girl", n=3, output_path=["girl1.png", "girl2.png", "girl3.png"], verbose=True)

  0s [INFO] {"output": {"task_status": "PENDING", "task_id": "551a73bd-88e7-4628-83c4-cc6e8b639929"}, "request_id": "238f7d2d-9911-9f72-b8ba-9e9fd0493881"}
  0s [INFO] 551a73bd-88e7-4628-83c4-cc6e8b639929: PENDING
 29s [IMAGE_URL] https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/32/20240918/1b61f1c0/077d351c-03e4-464b-b06c-2b835c4bd1a7-1.png?Expires=1726680122&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=BiIfdSxc9Ne0aV%2Bq%2FF3t704YGKU%3D
 29s [INFO] output image to girl1.png
 29s [IMAGE_URL] https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/ec/20240918/1b61f1c0/31d17a8b-bee4-492b-b4e4-f8d716588de6-1.png?Expires=1726680122&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=Tg1ER%2Bzna8i1I0ZcM9OaVFRnFmA%3D
 29s [INFO] output image to girl2.png
 29s [IMAGE_URL] https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/ee/20240918/1b61f1c0/158f1a32-e054-4110-94a4-37c8024c4530-1.png?Expires=1726680122&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=VgpR5160PUdp

''

In [2]:
from illufly.tools import Text2ImageWanx
from illufly.io import log, alog

w = Text2ImageWanx()
await alog(w, "a girl", n=3, verbose=True)

  0s [INFO] {"output": {"task_status": "PENDING", "task_id": "81c3c4df-15f9-43ca-a0d3-ca3518823302"}, "request_id": "4caaba4b-9d53-9736-87b7-bd1b1fc03224"}
  0s [INFO] 81c3c4df-15f9-43ca-a0d3-ca3518823302: PENDING
 29s [IMAGE_URL] https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/4a/20240918/1b61f1c0/03e365b2-4868-43c0-b9ad-9627ba85c6e0-1.png?Expires=1726680168&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=wTMUkv%2B0ATbJ5LiLZeEtX%2BFoBZg%3D
 30s [INFO] output image to 03e365b2-4868-43c0-b9ad-9627ba85c6e0-1_0.png
 30s [IMAGE_URL] https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/e9/20240918/1b61f1c0/d65c6ff4-819b-4951-b751-abdacc7f2d7c-1.png?Expires=1726680168&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=30aQH8q5kVUF%2FrWaL6H%2B%2FG6Pdv8%3D
 30s [INFO] output image to 03e365b2-4868-43c0-b9ad-9627ba85c6e0-1_1.png
 30s [IMAGE_URL] https://dashscope-result-hz.oss-cn-hangzhou.aliyuncs.com/1d/d8/20240918/522176a8/1f7b74ec-2bec-4c0d-9fa7-c53d6541545d-1.png?Expires

''

## Cosplay

In [3]:
import os
from urllib.parse import urlparse, unquote
parsed_url = urlparse("https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/viapi-video/2024-09-17/63309626-c9d7-48c6-9df4-ec579212a613/20240917184529907756_1_rpndhug9lk.jpg?Expires=1726656336&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=jlv8VajsiYWGmeRiqn%2F4Wah6zyg%3D")
os.path.basename(parsed_url.path)

'20240917184529907756_1_rpndhug9lk.jpg'

In [4]:
from illufly.tools import CosplayWanx
from illufly.io import log, alog

w = CosplayWanx()
log(w, verbose=True)

  1s [INFO] {"output": {"task_status": "PENDING", "task_id": "65b648c9-73a8-4b38-8154-7913254ccc30"}, "request_id": "c6de1abc-d965-9fce-8fc1-bbf21a50c836"}
  1s [INFO] 65b648c9-73a8-4b38-8154-7913254ccc30: PENDING
 10s [IMAGE_URL] https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/viapi-video/2024-09-18/8fecf20d-cc0f-4e12-8139-f127d3ebac23/20240918012300341467_1_25kptsg4mc.jpg?Expires=1726680186&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=W7fR3qmXkUMnpZyunOaWmU8xe7w%3D
 10s [INFO] output image to 20240918012300341467_1_25kptsg4mc.jpg
 10s [USAGE] {"image_count": 1}


''

In [1]:
from illufly.tools import CosplayWanx
from illufly.io import log, alog

w = CosplayWanx()
await alog(w, verbose=True)

  1s [INFO] {"output": {"task_status": "PENDING", "task_id": "1543b458-5e7c-4107-a183-650f7896d87d"}, "request_id": "4d4ec17b-bb48-9ebc-8f07-ffd21b1979d8"}
  1s [INFO] 1543b458-5e7c-4107-a183-650f7896d87d: PENDING
 10s [IMAGE_URL] https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/viapi-video/2024-09-18/d77476b6-012f-48f7-ad79-9e0905702d52/20240918014238320881_1_rs3nxbi9l4.jpg?Expires=1726681365&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=oigH4uGNO3ZZGK%2FzmJMB8rzLRjA%3D
 10s [INFO] output image to 20240918014238320881_1_rs3nxbi9l4.jpg
 10s [USAGE] {"image_count": 1}


''

## 看图说话

## Mermaid 图

## PlantUML 图

In [5]:
int(500 / 8)*8

496